In [ ]:
# Face Attendance: Webcam preview + basic face boxes (Haar)
import cv2
import os
from datetime import datetime

# Use OpenCV built-in cascade if local file not found
LOCAL_CASCADE = 'haarcascade_frontalface_default.xml'
if os.path.exists(LOCAL_CASCADE):
    CASCADE_PATH = LOCAL_CASCADE
else:
    CASCADE_PATH = os.path.join(cv2.data.haarcascades, 'haarcascade_frontalface_default.xml')
    if not os.path.exists(CASCADE_PATH):
        raise FileNotFoundError('Haar cascade tidak ditemukan (lokal maupun dari OpenCV).')

face_detector = cv2.CascadeClassifier(CASCADE_PATH)

cam = cv2.VideoCapture(0)
cam.set(3, 640)
cam.set(4, 480)
if not cam.isOpened():
    print("Error: Tidak dapat membuka kamera.")
else:
    print("Kamera aktif. Tekan 'q' untuk keluar.")

while cam.isOpened():
    ret, frame = cam.read()
    if not ret:
        print("Gagal mengambil frame")
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, "Face", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)
    cv2.imshow('Face Recognition (preview)', frame)
    k = cv2.waitKey(1) & 0xff
    if k == 27 or k == ord('q'):
        break

print("\nProgram selesai.")
cam.release()
cv2.destroyAllWindows()

[ERROR:0@610.369] global persistence.cpp:566 open Can't open file: 'haarcascade_frontalface_default.xml' in read mode


Kamera aktif.


error: OpenCV(4.12.0) /io/opencv/modules/objdetect/src/cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'detectMultiScale'


# 📚 Langkah kerja ML microservice (Enroll + Recognize)
Berikut langkah-langkah untuk membuat microservice ML untuk absensi siswa:
- Install dependencies
- Jalankan API (FastAPI)
- Enroll wajah siswa (dari file atau kamera)
- Recognize wajah secara real-time (via API atau langsung webcam)

In [ ]:
# 1) Install dependencies (opsional jika sudah ada)
import sys, subprocess, os
req = os.path.join(os.getcwd(), 'requirements.txt')
if os.path.exists(req):
    print('Installing requirements...')
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', req], check=False)
else:
    print('requirements.txt tidak ditemukan, lewati instalasi.')

## Jalankan API
API FastAPI:
- Endpoint `POST /enroll` untuk menambahkan wajah baru (`student_id`, `image`)
- Endpoint `POST /recognize` untuk mendeteksi wajah pada gambar
Jalankan server di terminal terpisah:
```bash
uvicorn src.ml.api:app --reload
```

In [ ]:
# 2) Helper untuk enroll dari folder known_faces/<label>/*.jpg
from importlib import import_module
bulk = import_module('src.ml.bulk_enroll')
bulk.main()
print('Bulk enroll selesai')

In [ ]:
# 3) Enroll satu gambar (contoh) via API
import requests, os
api = 'http://127.0.0.1:8000'
sample_path = None
for root, _, files in os.walk('known_faces'):
    for f in files:
        if f.lower().endswith(('.jpg','.jpeg','.png')):
            sample_path = os.path.join(root, f)
            break
    if sample_path: break
if sample_path:
    files = {'image': open(sample_path, 'rb')}
    data = {'student_id': os.path.basename(os.path.dirname(sample_path))}
    r = requests.post(api + '/enroll', files=files, data=data, timeout=30)
    print('Response:', r.status_code, r.text)
else:
    print('Tidak ditemukan contoh gambar di known_faces/')

In [ ]:
# 4) Recognize dari satu gambar (contoh) via API
import requests, os
api = 'http://127.0.0.1:8000'
test_path = sample_path
if test_path and os.path.exists(test_path):
    files = {'image': open(test_path, 'rb')}
    r = requests.post(api + '/recognize', files=files, timeout=30)
    print('Response:', r.status_code, r.text)
else:
    print('Tidak ada sample untuk test recognize')

## Real-time attendance (opsi)
Gunakan demo webcam langsung (tanpa API) untuk uji cepat:
```bash
python -m src.ml.cam_demo
```
Atau integrasikan dengan aplikasi client (web/desktop) yang
stream frame -> panggil endpoint `/recognize` setiap interval (mis. 1-2 fps) untuk mencatat absensi.